# 比赛背景
全国大学生智能汽车竞赛是以智能汽车为研究对象的创意性科技竞赛，是面向全国大学生的一种具有探索性工程的实践活动，是教育部倡导的大学生A类科技竞赛之一。竞赛以立足培养，重在参与，鼓励探索，追求卓越为指导思想，培养大学生的创意性科技竞赛能力。

2023年是百度完全模型竞速赛的第二年，分为线上资格赛、线下分区赛和全国总决赛三个阶段，组委会将综合考虑线上资格赛和线下分区赛成绩来进行全国总决赛名额的选拔，其中线上成绩占10%，线下成绩占90%。参赛学生必须在规定时间内使用百度开源深度学习平台飞桨进行模型的设计、训练和预测，不得使用其他相关平台、框架及任何飞桨中未包含的学习方法参赛。

AI Studio作为本次线上选拔赛的唯一指定训练平台，提供在线编程环境、免费GPU算力、海量开源算法和开放数据，帮助开发者快速创建和部署模型。报名参加完全模型组线上资格赛的学校将免费获得百度EdgeBoard计算板借用资格。

# 赛题说明
## 比赛数据
本次比赛提供8类共计4770张图像数据，采用矩形检测框标注方法。所用数据集请访问[官方|第十八届全国大学生智能汽车竞赛—百度完全模型组竞速赛-线上预选赛数据集](http://aistudio.baidu.com/aistudio/datasetdetail/191561)。具体分类标准如下：

![](https://ai-studio-static-online.cdn.bcebos.com/0e445c17f241414097243539374ce5a830ae0544e74948f2bfd7e15a516ef030)


## 比赛任务
本次比赛是计算机视觉中一个典型的目标检测任务，即从图像中获得特定目标的类别信息和位置信息（常用矩形检测框的坐标表示），一般使用深度学习模型完成。目标检测的效果如下图所示：

![](https://ai-studio-static-online.cdn.bcebos.com/264554aa5acd43449551208fc175200706827cc0d1864aa4b7915124b4c4296f)


## 评估标准
比赛要求选手提交时上传模型文件、运行代码等，会在评估平台上运行检测模型，对于图片的检测速度要求高于20fps，最终会使用表示检测精度的F1-Score进行排名。

![](https://ai-studio-static-online.cdn.bcebos.com/3890ea4f65904f6898f454ad4fbc68fd06f2fa743cf344f38a7b9f150b7bb4b6)


# PaddleDetection模型库
[PaddleDetection](https://github.com/PaddlePaddle/PaddleDetection)是一个基于PaddlePaddle的目标检测端到端开发套件，在提供丰富的模型组件和测试基准的同时，注重端到端的产业落地应用，通过打造产业级特色模型|工具、建设产业应用范例等手段，帮助开发者实现数据准备、模型选型、模型训练、模型部署的全流程打通，快速进行落地应用。

PP-YOLOE是基于PP-YOLOv2的卓越的单阶段Anchor-free模型，超越了多种流行的YOLO模型。PP-YOLOE避免了使用诸如Deformable Convolution或者Matrix NMS之类的特殊算子，以使其能轻松地部署在多种多样的硬件上。其使用大规模数据集obj365预训练模型进行预训练，可以在不同场景数据集上快速调优收敛。

![](https://user-images.githubusercontent.com/61035602/213202797-3a1b24f3-53c0-4094-bb31-db2f84438fbc.jpeg)

# 基线程序说明

1）小白也能上手：采用官方PaddlePaddle-GPU v2.2的环境，使用PaddleDetection套件行训练、推理、结果生成，是第一个完整包含检测及后处理全流程的基线程序；

2）流程规范，培养学生打算法赛的正确习惯：基线程序步骤清晰、注释丰富，旨在为参赛选手提供打榜流程上的全环节支持；

3）提高空间大，欢迎各路大佬魔改：除了官方发布的基线，每年都会有各路选手参与基线的魔改工作，开源自己的优质方案，期待你的AI Studio项目，发布时请带上“第十八届”、“完全模型组”等字样。



## (一) 比赛环境准备

In [ ]:
# 首先从git上下载PaddleDetection
%cd /home/aistudio/work
# 如果git clone的速度太慢，可以打开代理试试
#export https_proxy=http://172.19.57.45:3128
#export http_proxy=http://172.19.57.45:3128
!git clone https://github.com/PaddlePaddle/PaddleDetection.git

# #查看一下主目录下的文件
!ls /home/aistudio/work

In [ ]:
# 在AIStudio环境中安装相关依赖，requirements里面很多已经安装过了，只需要再安装pycocotools等少数依赖
%cd /home/aistudio/work/PaddleDetection
!nohup pip install pycocotools typeguard lap

In [ ]:
#运行一下在GPU环境下检测一张图片的程序，测试环境是否OK
!export CUDA_VISIBLE_DEVICES=0
!python tools/infer.py -c configs/ppyolo/ppyolo_r50vd_dcn_1x_coco.yml -o use_gpu=true weights=https://paddledet.bj.bcebos.com/models/ppyolo_r50vd_dcn_1x_coco.pdparams --infer_img=demo/000000014439.jpg

##  (二)  比赛数据准备
在Baseline中的data目录下，是用于进行目标检测的voc数据集格式的数据DatasetVocSASU_ForIcarM2023.zip，已经划分好了训练集和验证集(8:1)。选手也可以使用数据集中的random_split.py进行重新划分数据集。

In [ ]:
# 训练集和验证集数据解压缩至/home/aistudio/work/data_voc
!nohup unzip -d /home/aistudio/work/ /home/aistudio/data/data191561/DatasetVocSASU_ForIcarM2023.zip
!mkdir /home/aistudio/work/data_voc
!mv -f /home/aistudio/work/DatasetVocSASU_ForIcarM2023/DatasetVocSASU_ForIcarM2023/* /home/aistudio/work/data_voc/
!rm -rf /home/aistudio/work/DatasetVocSASU_ForIcarM2023

In [ ]:
#在/home/aistudio/work/PaddleDetection/dataset/下建立data_voc数据集软连接
!ln -s /home/aistudio/work/data_voc /home/aistudio/work/PaddleDetection/dataset/

##  (三)  检测模型训练及模型转换

#### A. 增加比赛的配置文件

In [ ]:
# 首先官方数据使用了voc标注的形式，为了将比赛的数据集与现有的区分开，这里复制voc.yml形成一个数据集配置文件data_voc.yml
!cp /home/aistudio/work/PaddleDetection/configs/datasets/voc.yml /home/aistudio/work/PaddleDetection/configs/datasets/data_voc.yml
# 注意修改data_voc.yml中的dataset_dir, anno_path, num_classes等字段
# 新的配置文件命名为data_voc.yml，存放路径为:PaddleDetection/configs/datasets/data_voc.yml
!ls /home/aistudio/work/PaddleDetection/configs/datasets

# 检测模型使用ppoloe plus s，在PaddleDetection/configs/ppyoloe/路径下，先复制为ppyoloe_plus_crn_s_80e_coco.yml为ppyoloe_plus_crn_s_80e_voc.yml
!cp /home/aistudio/work/PaddleDetection/configs/ppyoloe/ppyoloe_plus_crn_s_80e_coco.yml /home/aistudio/work/PaddleDetection/configs/ppyoloe/ppyoloe_plus_crn_s_80e_voc.yml
# 修改ppyoloe_plus_crn_s_80e_voc.yml中的data_config, weight等路径
!ls /home/aistudio/work/PaddleDetection/configs/ppyoloe/


#### B. 训练并保存模型

In [6]:
# 下面使用已经写好的配置文件，开始训练
# 基线程序中训练了个80epoch，并且已经保存了该模型，如果想跑流程的话，可以不用训练，直接进入到下一步转换模型步骤，本基线项目中保存了最终模型
#单卡
!python tools/train.py -c configs/ppyoloe/ppyoloe_plus_crn_s_80e_voc.yml --eval --amp --use_vdl=true --vdl_log_dir=vdl_dir/scalar
#多卡
#!python -m paddle.distributed.launch --gpus 0,1,2,3 tools/train.py -c configs/ppyoloe/ppyoloe_plus_crn_s_80e_voc.yml --eval --amp --use_vdl=true --vdl_log_dir=vdl_dir/scalar

#### C. 转换并保存静态图模型

In [7]:
# 训练完成后，模型将会被保存在output/ppyoloe_plus_crn_s_80e_voc/中，根据eval的效果，会保存出model_final.pdparams，这里使用这个模型进行静态图转换
%cd /home/aistudio/work/PaddleDetection
!python tools/export_model.py -c  configs/ppyoloe/ppyoloe_plus_crn_s_80e_voc.yml -o weights=output/ppyoloe_plus_crn_s_80e_voc/model_final.pdparams TestReader.fuse_normalize=true

In [8]:
# 转换好的静态图模型，将会保存在output_inference/ppyoloe_plus_crn_t_auxhead_300e_voc/文件夹中
!ls output_inference/ppyoloe_plus_crn_s_80e_voc/

截至到这里，检测模型的训练和转换已经完成

##  (四)  准备预测脚本和提交代码
本次比赛要求提交一个zip文件，里面包括了模型文件，训练和预测代码等，具体格式如下： 

| -- predict.py  
| -- train.py  
| -- PaddleDetection  
| -- model  
| | -- ppyoloxxx  
| | | -- infer_cfg.yml  
| | | -- model.pdiparams  
| | | -- model.pdiparams.info  
| | | -- model.pdmodel  

其中，预测代码需命名为predict.py，model目录不超过200M（不压缩），整体压缩包不超过1G。


In [10]:
# 首先退回到work目录下，我们准备提交必备的文件夹和文件
%cd /home/aistudio/work/

In [11]:
# 创建model文件夹（已经创建好的话，就不用再创建了）
!mkdir model

# 将训练脚本拷贝到work目录下，与model同级
!cp PaddleDetection/tools/train.py .

In [12]:
# 将检测模型拷贝到model文件夹中
!cp -r PaddleDetection/output_inference/ppyoloe_plus_crn_s_80e_voc/  model/
!ls model/

In [13]:
# 最后准备predict.py脚本进行预测（这里predict.py脚本已经写好，其中默认使用的模型就是model下面的ppyoloe_plus_crn_s_80e_voc，如果需要修改成为其他模型，请自行修改)
# predict.py脚本是根据paddledetection/deploy/python/infer.py修改而来，仅供参考，如想拿到好成绩，需要进一步优化
# predict.py中有一部分类别和处理是采用的默认值，需要参赛选手自行设计算法修改，否则成绩会比较低
# predict.py会根据data.txt文件中的图片进行预测，结果保存在result.json文件中，运行前请先解压数据集
!python predict.py data.txt result.json

In [14]:
# 代码打包
#打包前删除数据集软连接
!rm -rf /home/aistudio/work/PaddleDetection/dataset/data_voc
!rm -rf /home/aistudio/work/PaddleDetection/output_inference
!rm -rf /home/aistudio/work/PaddleDetection/vdl_dir
# 这里为了让压缩包更小，建议把PaddleDetection里面不用的模型备份出来,同时测评要求模型model在200M以内
# 然后从aistudio上下载打包好的submission.zip文件，就可以直接到官网提交啦！
!zip -r -q -o submission.zip model/ PaddleDetection/ train.py predict.py

## (五) 提交结果

最后将压缩文件提交到[评估平台](https://aistudio.baidu.com/aistudio/competition/detail/760/0/leaderboard)

![](https://ai-studio-static-online.cdn.bcebos.com/6f4112a3f4c8473d834ce2b1b688c60517ebed2bfc6446138826867b4c1e6d22)


## (六) 算法优化
选手可以从多个方面优化模型，包括但不限于以下思路：
1. 模型选型
2. 数据分析和数据增强
3. 训练调参
4. 后处理优化
5. 大模型压缩
.....


# 参考资料
1. [https://github.com/PaddlePaddle/PaddleDetection](https://github.com/PaddlePaddle/PaddleDetection)
2. [简介 PP-YOLOE+](https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.5/configs/ppyoloe/README_cn.md)
3. [模型详情 PP-YOLOE+](https://aistudio.baidu.com/aistudio/modelsdetail?modelId=33) 
4. [论文 PP-YOLOE: An evolved version of YOLO](https://arxiv.org/abs/2203.16250)



![](https://user-images.githubusercontent.com/61035602/220058003-d9ad2188-060e-4a81-a436-47c55c2e1ba1.png)
## [欢迎浏览PaddleDetection Github 首页](https://github.com/PaddlePaddle/PaddleDetection)

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 